In [1]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [2]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10']

In [3]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [4]:
ar.home()

In [5]:
stp = tools.stationary_touch_probe(ar)

# Initializing racks

In [6]:
# Tips
p1000_tip_rack = racks.consumables(rack_name='p1000_tips_1', rack_type='p1000_tips')
p200_tip_rack = racks.consumables(rack_name='p200_tips_1', rack_type='p200_tips')
p20_tip_rack = racks.consumables(rack_name='p20_tips_1', rack_type='p20_tips')
p1000_tip_rack.replaceConsumables()
# Some of the tips are missing from the box
p1000_tip_rack.removeConsumableItems([(4,7), (5,7), (7,7), (8, 6), (8, 7)])
p200_tip_rack.replaceConsumables()
p20_tip_rack.replaceConsumables()
# Tube racks
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
magep1 = racks.rack(rack_name='eppendorf_magnetic_1', rack_type='eppendorf_magnetic')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')
# Waste rack
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack')
# Have to provide slot numbers
rack96well = racks.rack(rack_name='96well_rack', rack_type='96wells_rack', x_slot=2, y_slot=3)
# Magnetic rack has the same geometry as normal rack, so the same rack type is used
magrack96well = racks.rack(rack_name='96well_rack_magnetic', rack_type='96wells_rack', x_slot=3, y_slot=3)

# Calibrating everyting

In [7]:
tp = tools.mobile_touch_probe.getTool(ar)
#tp = tools.mobile_touch_probe(ar)

In [8]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [40]:
xr, yr, zr = calibration.calibrateRack(tp, p1000_tip_rack)

In [43]:
xr, yr, zr = calibration.calibrateRack(tp, p20_tip_rack)

In [9]:
xr, yr, zr = calibration.calibrateRack(tp, p1000_tip_rack)
xr, yr, zr = calibration.calibrateRack(tp, p200_tip_rack)
xr, yr, zr = calibration.calibrateRack(tp, p20_tip_rack)
ar.move(z=400)
xr, yr, zr = calibration.calibrateRack(tp, ep1)
xr, yr, zr = calibration.calibrateRack(tp, magep1)
xr, yr, zr = calibration.calibrateRack(tp, rack96well)
xr, yr, zr = calibration.calibrateRack(tp, magrack96well)
xr, yr, zr = calibration.calibrateRack(tp, r50)
xr, yr, zr = calibration.calibrateRack(tp, waste_rack)

In [51]:
xr, yr, zr = calibration.calibrateRack(tp, ep1)
xr, yr, zr = calibration.calibrateRack(tp, magep1)

In [17]:
magep1.x_init_calibrat_dist_from_wall

2.0

In [50]:
ar.move(z=450)

In [29]:
xr, yr, zr = calibration.calibrateRack(tp, magep1)

In [10]:
tp.returnTool()

In [11]:
ar.home()

# Initializing samples

In [7]:
# Initializing samples (total 5 samples)
sample_list = [samples.sample(sample_name='t_ep_'+str(x), sample_type='eppendorf_tube') for x in range(5)]
[x[0].place(ep1, x[1], 3) for x in zip(sample_list, range(5))]

[None, None, None, None, None]

In [8]:
# Defining starting volume of each sample
vol_list = [50, 50, 40, 25, 25]

In [37]:
water = samples.sample(sample_name='water', sample_type='eppendorf_tube')
water.place(ep1, 7, 0)
water.setVolume(1300)

In [10]:
# Tube with magnetic beads
magbeads = samples.sample(sample_name='magbeads', sample_type='50mL_tube')
magbeads.place(r50, 0, 0)
# Assuming we have 20 mL of magnetic beads
magbeads.setVolume(20000)

In [11]:
# Tubes to transfer intermediate DNA into
upper_cut_samples = [samples.sample(sample_name='ucs_'+str(x), sample_type='eppendorf_tube') for x in range(5)]
# Placing samples into the initial rack
[x[0].place(ep1, x[1], 2) for x in zip(upper_cut_samples, range(5))]

[None, None, None, None, None]

In [12]:
# Empty tubes for elution
result_samples = [samples.sample(sample_name='result_'+str(x), sample_type='eppendorf_tube') for x in range(5)]
[x[0].place(ep1, x[1], 0) for x in zip(result_samples, range(5))]

[None, None, None, None, None]

In [13]:
# 80% ethanol tube
ethanol = samples.sample(sample_name='ethanol', sample_type='50mL_tube')
ethanol.place(r50, 2, 0)
ethanol.setVolume(50000)

In [14]:
# Waste tube
waste = samples.sample(sample_name='waste', sample_type='50mL_tube')
waste.place(r50, 1, 0)
waste.setVolume(0)

# Beads to vol recalculation

In [15]:
def calcBeadsVol(sample_vol, DNA_size_cutoff):
    multiplier = 4.99325349e-01 - 9.91043764e+00 / DNA_size_cutoff + 2.57585836e+04 / DNA_size_cutoff ** 2
    return sample_vol * multiplier

# DNA Size Cutoff

In [17]:
upper_cutoff_list = [1000, 500, 700, 350, 1000]
lower_cutoff_list = [250, 100, 100, 150, 200]

# Pre-filling tubes with "samples"

In [29]:
p200 = tools.pipettor.getTool(robot=ar, tool_name='p200_tool')

In [30]:
xt, yt, zt = calibration.calibrateTool(p200, stp)

In [31]:
xn, yn = p200_tip_rack.getNextConsumable()
p200.pickUpTip(p200_tip_rack, xn, yn, raise_z=300)

In [32]:
p200.distributeLiquid(water, sample_list, vol_list)

NameError: name 'dropTipToWaste' is not defined

In [33]:
ar.move(z=300)

In [38]:
test1_sample = samples.sample(sample_name='test', sample_type='eppendorf_tube')
test1_sample.place(magep1, 0, 0)

In [39]:
p200.moveLiquid(water, test1_sample, 200, blow_extra=True)
p200.moveLiquid(water, test1_sample, 200, blow_extra=True)
p200.moveLiquid(water, test1_sample, 200, blow_extra=True)
p200.moveLiquid(water, test1_sample, 200, blow_extra=True)

In [40]:
p200.moveLiquid(test1_sample, water, 200, blow_extra=True)
p200.moveLiquid(test1_sample, water, 200, blow_extra=True)
p200.moveLiquid(test1_sample, water, 200, blow_extra=True)
p200.uptakeAll(test1_sample)
p200.dispenseLiquid(water, 200, blow_extra=True)

In [18]:
# Estimating volume that needs to be added to the tubes
vol_to_add = [calcBeadsVol(x[0].getVolume(), x[1]) for x in zip(sample_list, upper_cutoff_list)]
vol_to_add

[0.0, 0.0, 0.0, 0.0, 0.0]

In [41]:
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack')

In [42]:
p200.dropTipToWaste(waste_rack, raise_z=400)

In [43]:
p200.returnTool()

In [65]:
ar.home()

# Moving tube from one rack to the other

In [45]:
gr = tools.mobile_gripper.getTool(robot=ar)

In [46]:
calibration.calibrateToolCustomPoints(tool=gr, stationary_probe=stp)

(94.8, 66.6, 515.9)

In [47]:
gr.moveSample(test1_sample, ep1, 4, 2)

# Moving several samples

In [48]:
for sample in sample_list:
    col_number = sample.sample_data['x_well']
    sample = gr.moveSample(sample, magep1, col_number, 1)

In [49]:
for sample in sample_list:
    col_number = sample.sample_data['x_well']
    sample = gr.moveSample(sample, ep1, col_number, 3)

In [63]:
gr.operateGripper(0)

In [60]:
# 80% ethanol tube
ethanol = samples.sample(sample_name='ethanol', sample_type='50mL_tube')
ethanol.place(r50, 0, 0)
ethanol.setVolume(50000)

In [61]:
gr.gripper_has_something = False

In [62]:
ethanol = gr.moveSample(ethanol, r50, 1, 0)
ethanol = gr.moveSample(ethanol, r50, 2, 0)
ethanol = gr.moveSample(ethanol, r50, 1, 1)
ethanol = gr.moveSample(ethanol, r50, 0, 0)

In [59]:
ar.move(z=300)

In [64]:
gr.returnTool()